<a href="https://colab.research.google.com/github/YasminaNasser/sattelite_images-segmentation/blob/main/satellite_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyngrok flask

In [4]:
!pip install Flask pyngrok tensorflow numpy pillow


In [5]:
!pip install segmentation_models

In [6]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm

Segmentation Models: using `tf.keras` framework.


In [7]:
from segmentation_models import Unet
from tensorflow.keras import Model
from tensorflow.keras.layers import Input
import tensorflow as tf

In [8]:
from segmentation_models import FPN
import tensorflow as tf
from keras.saving import register_keras_serializable

class CustomFPN14(Model):
    def __init__(self, input_shape=(128, 128, 14), backbone='resnet50'):
        super(CustomFPN14, self).__init__()

        # 14-channel base model (no weights yet)
        self.base = FPN(backbone_name=backbone, input_shape=input_shape,
                         encoder_weights=None, classes=1, activation='sigmoid')

        # Load pretrained 3-channel model
        base_weights = FPN(backbone_name=backbone, input_shape=(128, 128, 3),
                            encoder_weights='imagenet', classes=1, activation='sigmoid')

        # Transfer weights except first 3 layers
        for i in range(1, len(self.base.layers)):
            try:
                self.base.layers[i].set_weights(base_weights.layers[i].get_weights())
            except:
                print(f"Skipping layer {i}: {self.base.layers[i].name}")

        del base_weights

    def call(self, x, training=None):
        return self.base(x)

In [9]:

from flask import Flask, request, jsonify, send_file
import tensorflow as tf
import numpy as np
import io
from PIL import Image
from pyngrok import ngrok, conf
import os
from tifffile import imread as tif_imread

app = Flask(__name__)

model_weights_path = 'model_weights.weights.h5'
if not os.path.exists(model_weights_path):
    print(f"Warning: Model weights file '{model_weights_path}' not found.")
    print("The model will not be able to make a real prediction.")

try:
    model = CustomFPN14(input_shape=(128, 128, 14), backbone='efficientnetb5')
    model.build((None, 128, 128, 14))
    model.load_weights(model_weights_path)
    print("Model loaded successfully (or dummy model created).")
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

def preprocess_input(image):
    """
    Preprocesses the input image for the model.
    """
    image = tf.image.resize(image, (128, 128))
    image = tf.cast(image, tf.float32) / 255.0
    return image

def postprocess_mask(pred):
    """
    Converts the model's prediction into a PIL Image.
    """
    pred = tf.squeeze(pred)
    if not isinstance(pred, np.ndarray):
        pred = pred.numpy()
    pred = (pred * 255).astype(np.uint8)
    return Image.fromarray(pred)

HTML_CONTENT = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Model Predictor</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap" rel="stylesheet">
    <style>
        body {
            font-family: 'Inter', sans-serif;
        }
    </style>
</head>
<body class="bg-gray-100 flex items-center justify-center min-h-screen p-4">
    <div class="bg-white p-8 rounded-xl shadow-lg w-full max-w-md">
        <h1 class="text-3xl font-bold text-center text-gray-800 mb-6">Upload a .npy file</h1>
        <p class="text-center text-gray-600 mb-8">
            This application will process your file and return a segmentation mask.
        </p>

        <form id="uploadForm" class="space-y-6">
            <div>
                <label for="file-upload" class="block text-sm font-medium text-gray-700">
                    Choose file (128x128x14 .npy format)
                </label>
                <div class="mt-1 flex justify-center px-6 pt-5 pb-6 border-2 border-gray-300 border-dashed rounded-md">
                    <div class="space-y-1 text-center">
                        <svg class="mx-auto h-12 w-12 text-gray-400" stroke="currentColor" fill="none" viewBox="0 0 48 48" aria-hidden="true">
                            <path d="M28 8H12a4 4 0 00-4 4v20m32-12v8m0 0v8a4 4 0 01-4 4H12a4 4 0 01-4-4v-4m32-4l-3.1-3.1a4 4 0 00-5.4.1L24 24.3m32-12l-7.2 7.2m0 0L24 24.3m2.7-2.7l5.4-5.4a4 4 0 00.1-5.4L32 8" stroke-width="2" stroke-linecap="round" stroke-linejoin="round" />
                        </svg>
                        <div class="flex text-sm text-gray-600">
                            <label for="file-upload" class="relative cursor-pointer bg-white rounded-md font-medium text-indigo-600 hover:text-indigo-500 focus-within:outline-none focus-within:ring-2 focus-within:ring-offset-2 focus-within:ring-indigo-500">
                                <span>Upload a file</span>
                                <input id="file-upload" name="file" type="file" class="sr-only" required>
                            </label>
                            <p class="pl-1">or drag and drop</p>
                        </div>
                        <p class="text-xs text-gray-500" id="fileName">
                            No file chosen
                        </p>
                    </div>
                </div>
            </div>

            <div>
                <button type="submit" class="w-full flex justify-center py-2 px-4 border border-transparent rounded-md shadow-sm text-sm font-medium text-white bg-indigo-600 hover:bg-indigo-700 focus:outline-none focus:ring-2 focus:ring-offset-2 focus:ring-indigo-500 transition ease-in-out duration-150">
                    Predict Mask
                </button>
            </div>
        </form>

        <div id="resultContainer" class="mt-8 hidden">
            <h2 class="text-xl font-bold text-center text-gray-800 mb-4">Prediction Result</h2>
            <div id="loading" class="text-center hidden">
                <svg class="animate-spin h-8 w-8 mx-auto text-indigo-500" xmlns="http://www.w3.org/2000/svg" fill="none" viewBox="0 0 24 24">
                    <circle class="opacity-25" cx="12" cy="12" r="10" stroke="currentColor" stroke-width="4"></circle>
                    <path class="opacity-75" fill="currentColor" d="M4 12a8 8 0 018-8V0C5.373 0 0 5.373 0 12h4zm2 5.291A7.962 7.962 0 014 12H0c0 3.042 1.135 5.824 3 7.938l3-2.647z"></path>
                </svg>
                <p class="text-gray-500 mt-2">Making a prediction...</p>
            </div>
            <img id="resultImage" class="mt-4 mx-auto border-2 border-gray-300 rounded-lg shadow-md" src="" alt="Prediction Mask">
            <div id="error" class="mt-4 text-center text-red-600 font-medium hidden"></div>
        </div>
    </div>

    <script>
        const form = document.getElementById('uploadForm');
        const fileInput = document.getElementById('file-upload');
        const fileNameSpan = document.getElementById('fileName');
        const resultContainer = document.getElementById('resultContainer');
        const loadingSpinner = document.getElementById('loading');
        const resultImage = document.getElementById('resultImage');
        const errorDiv = document.getElementById('error');

        fileInput.addEventListener('change', (event) => {
            if (event.target.files.length > 0) {
                fileNameSpan.textContent = event.target.files[0].name;
            } else {
                fileNameSpan.textContent = 'No file chosen';
            }
        });

        form.addEventListener('submit', async (event) => {
            event.preventDefault();

            // Reset UI
            resultContainer.classList.add('hidden');
            loadingSpinner.classList.remove('hidden');
            errorDiv.classList.add('hidden');
            resultImage.src = '';

            const formData = new FormData(form);

            try {
                const response = await fetch('/predict', {
                    method: 'POST',
                    body: formData,
                });

                loadingSpinner.classList.add('hidden');

                if (response.ok) {
                    const blob = await response.blob();
                    const imageUrl = URL.createObjectURL(blob);
                    resultImage.src = imageUrl;
                    resultContainer.classList.remove('hidden');
                } else {
                    const errorData = await response.json();
                    errorDiv.textContent = `Error: ${errorData.error || 'Prediction failed'}`;
                    errorDiv.classList.remove('hidden');
                }
            } catch (err) {
                loadingSpinner.classList.add('hidden');
                errorDiv.textContent = `A network error occurred: ${err.message}`;
                errorDiv.classList.remove('hidden');
            }
        });
    </script>
</body>
</html>
"""

@app.route('/', methods=['GET'])
def index():

    return HTML_CONTENT

@app.route('/predict', methods=['POST'])

@app.route('/predict', methods=['POST'])
def predict():
    if 'file' not in request.files:
        return jsonify({'error': 'No file uploaded'}), 400

    file = request.files['file']
    filename = file.filename.lower()

    try:
        # Load the image depending on file type
        if filename.endswith('.tif') or filename.endswith('.tiff'):
            image = tif_imread(file)  # shape: (H, W, C)
        else:
            image = Image.open(file).convert('RGB')  # .png or others
            image = np.array(image)

        if image.ndim == 2:
            # Grayscale to RGB
            image = np.stack([image]*3, axis=-1)
        elif image.shape[-1] == 4:
            # RGBA to RGB
            image = image[..., :3]

        # Resize image to (128, 128)
        image = tf.image.resize(image, (128, 128)).numpy()

        # Normalize and expand to 14 channels
        image = image.astype(np.float32) / 255.0
        channels = image.shape[-1]

        if channels < 14:
            # Pad channels with zeros
            padding = np.zeros((128, 128, 14 - channels), dtype=np.float32)
            image = np.concatenate([image, padding], axis=-1)
        elif channels > 14:
            image = image[..., :14]

        image = tf.expand_dims(image, axis=0)  # Add batch dim

        prediction = model.predict(image)
        mask_image = postprocess_mask(prediction)

        img_io = io.BytesIO()
        mask_image.save(img_io, 'PNG')
        img_io.seek(0)
        return send_file(img_io, mimetype='image/png')

    except Exception as e:
        return jsonify({'error': f'Error processing image: {str(e)}'}), 500
if __name__ == '__main__':
    try:
        public_url = ngrok.connect(5000)
        print(f"Ngrok tunnel established. Your app is live at: {public_url}")

        app.run(port=5000)

    except Exception as e:
        print(f"An error occurred while starting the Flask app or ngrok: {e}")
        ngrok.kill()


/usr/local/lib/python3.11/dist-packages/keras_applications/imagenet_utils.py:276: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 14 input channels.
  warnings.warn(


Skipping layer 1: stem_conv
Error loading model: Unable to synchronously open file (truncated file: eof = 93323264, sblock->base_addr = 0, stored_eof = 383076280)
Ngrok tunnel established. Your app is live at: NgrokTunnel: "https://ed98c99a1c57.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Aug/2025 17:27:03] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Aug/2025 17:27:05] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step


INFO:werkzeug:127.0.0.1 - - [02/Aug/2025 17:27:25] "POST /predict HTTP/1.1" 200 -


In [10]:
!ngrok config add-authtoken 2zgjQwyTAsx0UqK0WNTluIRk75F_4eKRLJ2a3NR6R6FqJrhB7


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
